# Hand Written Digit Recognition using Neural Networks

This notebook is created in attemp to participate in a coursera challenge. The data is availble on kaggle.


As usual, import all neccasary stuff into the notebook.

In [21]:
import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn

First, I will start by loading the data in panda dataframe and do some EDA.

In [22]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
print(train.shape,test.shape)

((42000, 785), (28000, 784))


It appears that the test data has no label and seems to serve submission function. Cross validation and model evaluation will be done based on the data provided in Train.csv file only.
Next, split the train,test dataframe into X and Y dataframe where X store the data while Y store the label.

In [23]:
X = train.copy()
X = X.drop('label',axis=1)
Y = pd.DataFrame(train['label'])

In [24]:
print(X.shape,Y.shape)

((42000, 784), (42000, 1))


Split the data accordingly, test set of 40%

In [28]:
X_train, X_test,Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.4)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

((25200, 784), (25200, 1))
((16800, 784), (16800, 1))


In [29]:
%%bash
git add Handwritten_Neural.ipynb
git commit -m 'Add code of basic train test split'
git push

[master 35d40f9] Add code of basic train test split
 1 file changed, 93 insertions(+), 31 deletions(-)


git: 'puhs' is not a git command. See 'git --help'.

Did you mean this?
	push
